In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../Weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

In [ ]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot, hessian_convergence_block_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
# # Single Linkage Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [0.01, 0.01]
# MODEL_PATH = osp.join('../../examples/' + 'single_linkage.obj')
# SUBDIVISION_RESOLUTION = 5

In [ ]:
# # Sphere to hemoglobin sequence
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.01, 0.1]
# ISO_CS = [0.04, 0.04]
# MODEL_PATH = osp.join(weaving_dir + 'models/sphere_to_hemoglobin/5.obj')
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.05
# MODEL_NAME = './results/sphere_to_hemoglobin_5'

In [ ]:
import py_newton_optimizer
linkage = elastic_rods.SurfaceAttractedLinkage(SURFACE_PATH, True, MODEL_PATH, SUBDIVISION_RESOLUTION, False, InterleavingType.weaving)
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.updateSourceFrame()
linkage.set_design_parameter_config(True, True)
linkage.set_attraction_tgt_joint_weight(0)
linkage.set_holdClosestPointsFixed(False)
with so():
    opts = py_newton_optimizer.NewtonOptimizerOptions()
    opts.niter = 100
    opts.verbose = 10
    elastic_rods.compute_equilibrium(linkage, options = opts)

In [ ]:
# linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Attraction)

In [ ]:
view = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)

In [ ]:
# view.show()

In [ ]:
# with open('./closest_point.obj', 'w') as f:
#     pt_list = linkage.get_linkage_closest_point()
#     for i in range(int(len(pt_list)/3)):
#         f.write('v {} {} {}\n'.format(pt_list[3*i], pt_list[3*i + 1], pt_list[3*i + 2]))

In [ ]:
# with open('./centerline.obj', 'w') as f:
#     pt_list = linkage.centerLinePositions()
#     for i in range(int(len(pt_list)/3)):
#         f.write('v {} {} {}\n'.format(pt_list[3*i], pt_list[3*i + 1], pt_list[3*i + 2]))

In [ ]:
direction = np.zeros(linkage.numDoF())
direction[1] = 1
direction = np.random.normal(-1, 1, linkage.numDoF())

In [ ]:
np.linalg.norm(linkage.gradient())

In [ ]:
variableDP = False
energyType = elastic_rods.SurfaceAttractionEnergyType.Attraction

In [ ]:
# fd_gradient_test(linkage, 1e-4, etype=elastic_rods.SurfaceAttractionEnergyType.Attraction, direction=direction, variableDesignParameters=variableDP)

In [ ]:
# fd_gradient_test(linkage, 1e-2, etype=elastic_rods.SurfaceAttractionEnergyType.Attraction, direction=direction, variableDesignParameters=variableDP)

In [ ]:
linkage.set_holdClosestPointsFixed(False)

In [ ]:
linkage.set_use_centerline(True)

In [ ]:
linkage.set_attraction_tgt_joint_weight(0)

In [ ]:
open('closest_pt_stats.txt', 'w').close()
gradient_convergence_plot(linkage, minStepSize=1e-5, maxStepSize=1e-1, variableDesignParameters = variableDP, etype = elastic_rods.SurfaceAttractionEnergyType.Attraction, direction = direction)

In [ ]:
# # plot closest point stats
# with open(osp.join("closest_pt_stats.txt")) as f:
#     minStepSize=1e-5
#     maxStepSize=1e-1
#     stats = np.array([[int(x) for x in line.strip().split(' ')] for line in f.readlines()])[::2][::2]
#     epsilons = np.logspace(np.log10(minStepSize), np.log10(maxStepSize), 100)
#     labels = ['Faces', 'Edges', 'Vertices']
#     fig = plt.figure()
#     for i in range(3):
#         ax = fig.add_subplot(3, 1, i+1)
#         plt.plot(epsilons, stats[:, i], label=labels[i])
#         ax.set_xscale('log')
#         plt.legend()
#         plt.tight_layout()
#     plt.savefig('closest_pt_projection.png', dpi = 300)
#     plt.show()

In [ ]:
save_dof = linkage.getDoFs()

In [ ]:
linkage.setDoFs(linkage.getDoFs() + np.random.normal(-1e2, 1e2, linkage.numDoF()))

In [ ]:
linkage.setDoFs(save_dof)

In [ ]:
open('closest_pt_stats.txt', 'w').close()
hessian_convergence_plot(linkage, minStepSize=1e-5, maxStepSize=1e-1, variableDesignParameters = variableDP, etype = elastic_rods.SurfaceAttractionEnergyType.Attraction, direction = direction)

In [ ]:
grad = linkage.gradient(variableDesignParameters = variableDP)

In [ ]:
jv_offset = linkage.dofOffsetForJoint(0)
rk_offset = linkage.numDoF()
frl_offset = rk_offset + linkage.numRestKappaVars()
jrl_offset = frl_offset + linkage.numFreeRestLengths()

In [ ]:
linkage.numJoints()

In [ ]:
var_types = ['FULL', 'RV', 'JV', 'RK', 'FRL', 'JRL']
var_indices = {'FULL': range(0, linkage.numExtendedDoF()),
                 'RV': range(0, jv_offset),
                'JV' : range(jv_offset, rk_offset),
                 'RK': range(rk_offset, frl_offset),
                'FRL': range(frl_offset, jrl_offset),
                'JRL': range(jrl_offset, linkage.numExtendedDoF())
              }

In [ ]:
# block_error('VAR', 'RK', eps=1e-6)
print(var_indices)

In [ ]:
direction = np.zeros(linkage.numExtendedDoF())
print(linkage.numExtendedDoF())
direction[var_indices['RK']] = np.random.normal(0, 1, len(var_indices['RK']))

In [ ]:
# def decompose_gradient(g):
#     return {t: g[var_indices[t]] for t in var_types}

# def log_hessian_error(fd_delta_gradient, analytic_delta_gradient):
#     fd_components = decompose_gradient(fd_delta_gradient)
#     an_components = decompose_gradient(analytic_delta_gradient)
#     for k in fd_components:
#         print(f'{k}\t{la.norm(fd_components[k] - an_components[k]) / la.norm(an_components[k])}')

# stepSize = 1e-5
# fd_delta_grad, an_delta_grad = fd_hessian_test(linkage, stepSize, etype = elastic_rods.EnergyType.Full, direction = direction, variableDesignParameters = variableDP)
# log_hessian_error(fd_delta_grad, an_delta_grad)

In [ ]:
# hessian_convergence_block_plot(linkage, var_types, var_indices, variableDesignParameters=variableDP)

In [ ]:
# hs = single_linkage.hessianSparsityPattern(variableDesignParameters = True)

In [ ]:
# tm = hs.getTripletMatrix()

In [ ]:
# tm.dump('test.txt')

In [ ]:
# sparsity_pattern = np.zeros((115, 115, 3))

# with open ('test.txt', 'r') as f:
#     content = f.readlines()
#     for line in content:
#         line = line.strip()
#         line = line.split('\t')
#         i = int(line[0])
#         j = int(line[1])
#         if j < 73:
#             sparsity_pattern[int(line[0])][int(line[1])] = np.array([255, 255, 255])
#         else:
#             sparsity_pattern[int(line[0])][int(line[1])] = np.array([255, 0, 0])
            
# sparsity_pattern[0][79] = np.array([0, 255, 0])
                